In [1]:
import pandas as pd
import numpy as np
import feather

from tqdm import tqdm

In [3]:
from feature_gen import add_weekly_trends_features
from outliers import remove_outliers

In [4]:
df_pays = feather.read_dataframe('data/df_pays_na_test.feather')

In [6]:
shops = df_pays.shop_id.unique()
shops = sorted(shops)

dfs = []

for i in tqdm(shops):
    df_shop = df_pays[df_pays.shop_id == i].reset_index(drop=1)
    remove_outliers(df_shop)
    add_weekly_trends_features(df_shop)
    dfs.append(df_shop)

100%|██████████| 2000/2000 [1:07:41<00:00,  1.54s/it]


In [7]:
truncated_dfs = []

for df in dfs:
    df = df.iloc[7 * 2 * 3:]
    truncated_dfs.append(df)

In [8]:
df_features = pd.concat(truncated_dfs).reset_index(drop=1)

In [9]:
for c in df_features.columns:
    if 'trend' not in c or 'coef' in c:
        continue
    print(c)
    df_features.loc[df_features[c] <= 1, c] = 1

weekly_trend
biweekly_trend
biweekly_dow_trend
biweekly_weekend_trend


In [10]:
feather.write_dataframe(df_features, 'features/weekly_trends_features_noout.feather')